In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Scraping

In [3]:
def scrape_page(page_n):
    url = f"https://www.quandoo.fi/en/helsinki?districtFilter=3637&bookable=true&onlySpecialOffers=false&page={page_n}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rest_cards = soup.find_all(attrs={"data-qa": "merchant-card"})

    rest_names = [card.find('h3').text.strip() for card in rest_cards]
    rest_locations = [card.find(attrs={"data-qa": "merchant-location"}).text.strip() for card in rest_cards]
    # "Located in ... area" part removed in Location column.
    rest_locations = [location.replace('Located at ','').replace('area','') for location in rest_locations]
    
    rest_cuisines = [card.find(attrs={"data-qa": "merchant-card-cuisine"}).text.strip() for card in rest_cards]
    #Cleaned Cuisine's column data
    rest_cuisines = [cuisine.replace(" Restaurant","") for cuisine in rest_cuisines]

    rest_meals_boxes = [card.find(attrs={"data-qa": "merchant-meal"}) for card in rest_cards]
    rest_meals = [re.sub(r'^\s*Meals:\s*', '', box.text.strip()) if box else None for box in rest_meals_boxes]
    
    rest_rating_boxes = [card.find(attrs={"data-qa": "reviews-score"}) for card in rest_cards]
    rest_ratings = [float(box.text.split('/')[0].strip()) if box else None for box in rest_rating_boxes]

    rest_review_boxes = [card.find(class_="sc-1atis9w-3 dfyExP") for card in rest_cards]
    rest_review_counts = [int(box.text.split()[0].strip()) if box else None for box in rest_review_boxes]

    rest_price_boxes = [card.find(class_=re.compile(r'.*price-indicator')) for card in rest_cards]
    rest_price_levels = [len(box.find_all(class_=re.compile(r'.*oGCHK'))) for box in rest_price_boxes]

    rest_page_urls = [card.find('a')['href'] for card in rest_cards]

    df = pd.DataFrame({'Name': rest_names, 
                    'Location': rest_locations,
                    'Cuisine': rest_cuisines,
                    'Meals': rest_meals,
                    'Price Level (out of 4)': rest_price_levels,
                    'Rating (out of 6)': rest_ratings,
                    'Review Count': rest_review_counts,
                    'Page URL': rest_page_urls
                    })

    return df

In [4]:
restaurant_data = pd.DataFrame()

for page_n in range(1, 15):
    page_data = scrape_page(page_n)
    restaurant_data = pd.concat([restaurant_data, page_data], ignore_index=True)

restaurant_data

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
0,Luovuus kukkii kaaoksesta,Kaartinkaupunki,International,Dinner,3,5.8,470.0,/en/place/luovuus-kukkii-kaaoksesta-90397/menu
1,Ravintola MyStuu,Punavuori,Swiss,"Lunch, Dessert, Dinner",3,5.7,293.0,/en/place/ravintola-my-stuu-98898/menu
2,Gaucho,City Centre,Brazilian,Dinner,3,5.6,1122.0,/en/place/gaucho-105125/menu
3,Finlandia Caviar,City Centre,Gourmet,"Lunch, Dinner",3,5.6,143.0,/en/place/finlandia-caviar-15896/menu
4,Restaurant Armenian House,Kamppi,International,"Lunch, Dinner",2,5.6,317.0,/en/place/armenian-house-55148/menu
...,...,...,...,...,...,...,...,...
337,Lie Mi Lunch Kitchen,Ruoholahti,Vietnamese,"Lunch, Brunch",2,NaN,NaN,/en/place/lie-mi-lunch-kitchen-106571/menu
338,Mad Finn Brewing Co. Taproom Helsinki,Sompasaari,Pizza,Dinner,2,6.0,2.0,/en/place/mad-finn-brewing-co-taproom-helsinki...
339,Kahvila Mutteri,Lauttasaari,Dessert,Cake & Coffee,2,NaN,NaN,/en/place/kahvila-mutteri-100773/menu
340,Merisali - Hilton Kalastajatorppa,Munkkiniemi,Scandinavian,"Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...


In [5]:
restaurant_data.to_csv("restaurant_data.csv", sep="\t")

## Prettifying

In [7]:
data = pd.read_csv("restaurant_data.csv", sep="\t", index_col=0)
data.sample(10)

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
206,DIF Döner Ruoholahti,Ruoholahti,Kebab,"Lunch, Dinner, Sunday lunch",2,4.7,10.0,/en/place/dif-doner-95525/menu
43,Mizore Sushi & Sticks,Sörnäinen,Sushi,"Lunch, Dinner",3,5.1,247.0,/en/place/mizore-sushi-sticks-106386/menu
143,Peloton Cycling Eatery,Kallio,International,"Lunch, Dessert, Brunch, Dinner",3,5.7,24.0,/en/place/peloton-cycling-eatery-104752/menu
308,Ragazza Pasta Bar,Vuosaari,Pasta,"Lunch, Brunch, Dinner",3,5.2,12.0,/en/place/ragazza-pasta-bar-107135/menu
47,Retro Enoteca,Kaisaniemi,Italian,Dinner,2,5.1,210.0,/en/place/retro-enoteca-103875/menu
106,Tandoori Flames,Vallila,Indian,"Lunch, Dinner, Sunday lunch",2,5.7,38.0,/en/place/tandoori-flames-92432/menu
228,Uyghur Noodle House Kamppi,Kamppi,Asian,"Lunch, Dinner",1,3.8,6.0,/en/place/uyghur-noodle-house-kamppi-109171/menu
232,Amex Exclusive Lunch: Pastis,Kaartinkaupunki,French,NaN,3,5.0,2.0,/en/place/amex-exclusive-lunch-pastis-104118/a...
177,Cafe Elo,Malminkartano,Eat & Drink,"Breakfast, Lunch, Brunch, Cake & Coffee",2,4.9,73.0,/en/place/cafe-elo-98790/menu
55,Ristorante Il Trio,Töölö,Italian,"Lunch, Dessert, Dinner",3,5.2,379.0,/en/place/ristorante-il-trio-54885/menu
